In [10]:
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
#import imblearn
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold

In [11]:
train = pd.read_csv("pre_processed_train.csv")
test = pd.read_csv("pre_processed_test.csv")

In [12]:
train

,Unnamed: 0,TransactionID,isFraud,TransactionAmt,card1,card2,card3,card5,addr1,dist1,...,M4_M3,M5_F,M5_N,M5_T,M6_F,M6_N,M6_T,M8_F,M8_N,M8_T
0,0,3424175,0,7.000000,5033.0,269.0,150.0,224.0,330.0,2.449490,...,0,0,0,1,1,0,0,0,1,0
1,1,3206179,0,7.612490,4527.0,376.0,150.0,226.0,441.0,5.986395,...,1,0,1,0,1,0,0,0,1,0
2,2,3410455,0,10.768008,2772.0,512.0,150.0,226.0,143.0,3.741657,...,1,0,1,0,0,0,1,1,0,0
3,3,3058578,0,10.000000,14290.0,512.0,150.0,226.0,177.0,5.986395,...,1,0,1,0,0,1,0,0,1,0
4,4,3242131,0,9.746794,12839.0,321.0,150.0,226.0,220.0,2.449490,...,0,1,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442900,442900,3020019,0,5.385165,3277.0,111.0,150.0,226.0,231.0,5.986395,...,0,0,0,1,1,0,0,0,1,0
442901,442901,3546388,0,9.848858,18132.0,567.0,150.0,117.0,272.0,2.828427,...,0,0,0,1,1,0,0,0,1,0
442902,442902,3087445,0,10.000000,6019.0,583.0,150.0,226.0,441.0,5.986395,...,1,0,1,0,0,1,0,0,1,0
442903,442903,3066953,0,5.000000,10737.0,399.0,150.0,118.0,204.0,5.986395,...,1,0,1,0,0,1,0,0,1,0


In [13]:
t1 = train

In [14]:
ytrain = train["isFraud"]
train.drop(axis = 1, labels = ["isFraud", "Unnamed: 0"], inplace = True)

In [15]:
test.drop(axis = 1, labels = ["Unnamed: 0"], inplace = True)

In [16]:
over = SMOTE(sampling_strategy = 0.05)
under = RandomUnderSampler(sampling_strategy = 0.8)
steps = [('o', over),('u',under)]
pipeline = Pipeline(steps = steps)
train1, ytrain1 = pipeline.fit_resample(train, ytrain)

In [17]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
X_train = scale.fit_transform(train1)
X_test = scale.transform(test) 

In [18]:
param_grid = {
    'base_estimator__max_depth' : [1, 2, 3, 4, 5],
    'max_samples' : [0.05, 0.1, 0.2, 0.5]
}

clf = GridSearchCV(BaggingClassifier(DecisionTreeClassifier(),
                                     n_estimators = 100, max_features = 0.5),
                   param_grid, scoring = 'roc_auc')
clf.fit(X_train, ytrain1)

GridSearchCV(estimator=BaggingClassifier(base_estimator=DecisionTreeClassifier(),
                                         max_features=0.5, n_estimators=100),
             param_grid={'base_estimator__max_depth': [1, 2, 3, 4, 5],
                         'max_samples': [0.05, 0.1, 0.2, 0.5]},
             scoring='roc_auc')

In [19]:
print("Tuned Hyperparameters :", clf.best_params_)
print("Accuracy :",clf.best_score_)

Tuned Hyperparameters : {'base_estimator__max_depth': 5, 'max_samples': 0.5}
Accuracy : 0.8692407067890123


In [20]:
classifier = BaggingClassifier(base_estimator=DecisionTreeClassifier(), max_features=0.5, n_estimators=100)
classifier.fit(X_train, ytrain1)
y_pred1 = classifier.predict(X_test)

In [21]:
y_pred1.sum()

13477

In [22]:
import csv
with open('bagging.csv', 'w', newline = '') as file:
    writer = csv.writer(file)
    writer.writerow(["Id", "isFraud"])
    idx = 0
    for i in y_pred1:
        writer.writerow([idx, i])
        idx = idx + 1